In [ ]:
from keras.models import load_model
import glob
from keras.models import Sequential
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from numpy import expand_dims
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt
import shutil
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [ ]:
#### 이 셀은 밑에 아래 validation_matrix셀 실행 시, input ~~ cannot retrieve이런 식의 에러가 뜨면 실행해주세요. 그게 드라이브에서 불러오면 뭐가 불안해서 나는 에러라고합니다.
import tarfile
fname = '/content/drive/My Drive/Data_Augmentation.tar'  # 압축 파일을 지정해주고   
ap = tarfile.open(fname)      # 열어줍니다. 

ap.extractall('/content/Data')         # 그리고는 압축을 풀어줍니다. 
# () 안에는 풀고 싶은 경로를 넣어주면 되요. 비워둘 경우 현재 경로에 압축 풉니다. 

ap.close()  

fname = '/content/drive/My Drive/augmented_validation.tar'  # 압축 파일을 지정해주고   
ap = tarfile.open(fname)      # 열어줍니다. 

ap.extractall('/content/Data')         # 그리고는 압축을 풀어줍니다. 
# () 안에는 풀고 싶은 경로를 넣어주면 되요. 비워둘 경우 현재 경로에 압축 풉니다. 

ap.close()  

In [ ]:
original_model = load_model(model_path)
original_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, None, None, 2048)  23587712  
_________________________________________________________________
global_average_pooling2d_4 ( (None, 2048)              0         
_________________________________________________________________
sequential_7 (Sequential)    (None, 91)                547931    
Total params: 24,135,643
Trainable params: 5,013,595
Non-trainable params: 19,122,048
_________________________________________________________________


In [ ]:
from keras import applications

여기서 부터 보면 됌!!!

In [ ]:
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
model_path = "/content/Data/fine_tuned_model_totalV2.h5"
validation_data_dir = "/content/Data/augmented_validation"  # 0,1 과 같은 카테고리 번호 전까지

#### 버전 1 : ResNet50통과 후 GlobalAveragePooling2D까지만 한 벡터 활용
def making_validation_matrix(validation_data_dir, model_path):
    ## making vectorizing model
    vectorizing_model = Sequential()
    model = load_model(model_path)
    for layer in model.layers[:2]:
        vectorizing_model.add(layer)
    
    ## making validation matrix
    datagen = ImageDataGenerator(rescale=1. / 255)
    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode=None,
        shuffle=False)
    validation_matrix = vectorizing_model.predict_generator(generator)
    
    return vectorizing_model, validation_matrix

with tf.device('/device:GPU:0'):
    vectorizing_model, validation_matrix = making_validation_matrix(validation_data_dir)

In [ ]:
validation_matrix.shape

(10101, 2048)

In [ ]:
#vectorizing_model.save("/content/drive/My Drive/vectorizing_model.h5")
np.save("/content/drive/Shared drives/선빵팀 :) 2/validation_matrix/resnet50_matrix.npy",validation_matrix)

In [ ]:
def original(img_path):
    img = load_img(img_path)
    img = img_to_array(img)
    bgr_image = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
    return bgr_image

In [ ]:
# adaptive threshold만 적용
def threshold(imgpath):
    # grayscale 변환
    img = load_img(imgpath)
    img = img_to_array(img)
    gray_image = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #(a,b) -> 2차원으로 변환
    gray_image = gray_image.astype('uint8')
    
    # adaptive threshold 적용
    th = cv2.adaptiveThreshold(gray_image,
                               255,
                               cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                               cv2.THRESH_BINARY,
                               99,
                               15)
    
    # bgr 변환
    bgr_image = cv2.cvtColor(th,cv2.COLOR_GRAY2BGR)
    return bgr_image

In [ ]:
# 이미지 대비 향상 -> adaptive threshold
def threshold_sole(imgpath):
    # grayscale 변환
    img = load_img(imgpath)
    img = img_to_array(img)
    gray_image = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #(a,b) -> 2차원으로 변환
    gray_image = gray_image.astype('uint8')
    
    # 이미지 대비를 향상
    image_enhanced = cv2.equalizeHist(gray_image)
    
    # Adaptive Thresholding 적용 
    max_output_value = 255   # 출력 픽셀 강도의 최대값
    neighborhood_size = 99
    subtract_from_mean = 15
    image_binarized = cv2.adaptiveThreshold(image_enhanced,
                                          255,
                                          cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                          cv2.THRESH_BINARY,
                                          99,
                                          15)
    
    # bgr 변환
    bgr_image = cv2.cvtColor(image_binarized,cv2.COLOR_GRAY2BGR)  
    return bgr_image

In [ ]:
# morphology -> adaptive threshold
def morph_threshold(imgpath):
    # graysacle
    img = load_img(imgpath)
    img = img_to_array(img)
    gray_image = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray_image = gray_image.astype('uint8')
    
    # morph gradient
    kernel = np.ones((2,2), np.uint8)
    img2_grad = cv2.morphologyEx(gray_image, cv2.MORPH_GRADIENT, kernel)
    
    # adaptive threshold
    th = cv2.adaptiveThreshold(img2_grad, 
                               255, 
                               cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                               cv2.THRESH_BINARY,
                               99,
                               15)
    
    # bgr 변환
    bgr_image = cv2.cvtColor(th,cv2.COLOR_GRAY2BGR) # result함수 돌리기 위해 3차원으로 변환 
    return bgr_image

In [ ]:
# morph_gradient -> 이미지 대비 향상 -> adaptive threshold
def morph_threshold_sole(imgpath):
    #grayscale
    img = load_img(imgpath)
    img = img_to_array(img)
    gray_image = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray_image = gray_image.astype('uint8')
    
    # 이미지 대비를 향상
    image_enhanced = cv2.equalizeHist(gray_image)
    
    # morph_gradient
    kernel = np.ones((2,2), np.uint8)
    img2_grad = cv2.morphologyEx(image_enhanced, cv2.MORPH_GRADIENT, kernel)
    
    # Adaptive Thresholding 적용 
    image_binarized = cv2.adaptiveThreshold(img2_grad,
                                          255,
                                          cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                          cv2.THRESH_BINARY,
                                          99,
                                          15)
    
    # bgr 변환
    bgr_image = cv2.cvtColor(image_binarized,cv2.COLOR_GRAY2BGR)

    
    return bgr_image

In [ ]:
from skimage.transform import resize

In [ ]:
#comparison_img_path = "/content/Data/augmented_validation/15/15_0_1275.jpeg"
#label_validate = np.load("/content/drive/My Drive/label_validate.npy")

def model_sim_performance(vectorizing_model,comparison_img_path, validation_matrix, label_validate):
  # making img_matrix ( by concatenate original img vector and validation_matrix)
  comparison_img_category = int(comparison_img_path.split("/")[-1].split(" ")[0])  ## 이거는 img_path에서 카테고리 번호에 해당하는 부분이 어딘지를 찾아서 적절히 바꿔 주어야 함.
  
  datagen = ImageDataGenerator(rescale=1. /255)
  function = original # 위의 전처리 함수들 중 본인이 맡은 함수로 선택.
  img = function(comparison_img_path)
  x = expand_dims(img,0)
  generator = datagen.flow(x, y = None, batch_size = 1, shuffle=False, sample_weight=None)
  img_vector = vectorizing_model.predict_generator(generator, steps=1)
  
  matrix = np.concatenate((img_vector,validation_matrix))

  # calculate cosine_similarity
  cosine_sim_vector = cosine_similarity(matrix)[0][1:] # 자기자신을 제외한 나머지 벡터들과의 코사인 유사도 계산.


  # check model performance with similarity
  top100_index = cosine_sim_vector.argsort()[-100:]
  top100_categories = label_validate[top100_index]
  performance = (np.sum(top100_categories == comparison_img_category))

  return performance

#model_performance = model_sim_performance(vectorizing_model, comparison_img_path, validation_matrix, label_validate)
#print(model_performance)

In [ ]:
comparison_img_path = "/content/drive/Shared drives/선빵팀 :) 2/TuningStar_cate/1 (1).jpg"
validation_matrix = np.load("/content/drive/Shared drives/선빵팀 :) 2/validation_matrix/validation_matrix_morph_threshold_sole.npy")
label_validate = np.load("/content/drive/My Drive/label_validate.npy")
vectorizing_model = load_model("/content/drive/Shared drives/선빵팀 :) 2/Model/vectorizing_model_morph_threshold_sole.h5")

model_performance = model_sim_performance(vectorizing_model,comparison_img_path, validation_matrix, label_validate)
print(model_performance)

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


92


In [ ]:
import glob
import numpy as np
from tqdm.notebook import tqdm

with tf.device("/device:GPU:0"):
  model_performance_list=[]
  wheel_pathes=glob.glob("/content/drive/Shared drives/선빵팀 :) 2/TuningStar_cate/*.jpg")
  vectorizing_model = vectorizing_model
  label_validate = np.load("/content/drive/My Drive/label_validate.npy")
  validation_matrix = validation_matrix
  for comparison_img_path in tqdm(wheel_pathes):
    model_performance = model_sim_performance(vectorizing_model,comparison_img_path, validation_matrix, label_validate)
    model_performance_list.append(model_performance)

  performance_array = np.array(model_performance_list)
  print(np.mean(performance_array))

In [ ]:
# original : 44.98
# Adaptive Threshold : 67.30
# Morphology_equalizeHist_Adaptive : 48.75
# equalizeHist_Adaptive Threshold : 48.20
# Morphology_Adaptive Threshold : 41.03